In [ ]:
# Re-import necessary libraries after execution state reset
import pandas as pd
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, NonNegativeReals, Binary, SolverFactory, minimize, value

Imagine we have two coal plants in 2025, one of 20MW and another 
of 40MW and a wind plant of 10MW. We want to have in 2030 20MW of
coal and 60MW of wind.  And in 2035 we want 0MW of coal and 100 MW of wind

(used different numbers)

In [ ]:
# Re-import necessary libraries after execution state reset
import pandas as pd
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, NonNegativeReals, Binary, SolverFactory, minimize, value
# Define the optimisation model
model = ConcreteModel()
# Define time steps
time_steps = ["t1", "t2"]
# Define updated non-renewable plants with initial capacities
coal_plants = {
    "C1": 20,  # Coal Plant
    "C2": 40,  # Coal Plant
}
# Define renewable plants with initial capacities
wind_plants = {
    "W1": 10,  # Wind
    "W2": 0,  # Wind
    "W3": 0,  # Wind
    "W4": 0,  # Wind
    "W4": 0,  # Wind
    "W5": 0,  # Wind
    "W6": 0,  # Wind
    "W7": 0  # Wind
}
# Mapping renewable plant IDs to their type
plant_types = {
    "C1": "coal",
    "C2": "coal",
    "W1": "wind",
    "W2": "wind",
    "W3": "wind",
    "W4": "wind",
    "W5": "wind",
    "W6": "wind",
    "W7": "wind",
}
# Define unit repurposing costs per MW
repurposing_costs_per_mw = {
    "coal": {"wind": 50}
}
# Decommissioning costs per MW
decommissioning_costs_per_mw = {
    "coal": 10,
}
# Energy demand per time step
coal_energy_demand = {"t1": 10, "t2": 0}
wind_energy_demand = {"t1": 10, "t2": 50}

# Define decision variables
# Decommissioning actions
model.decom_coal = Var(coal_plants.keys(), time_steps, domain=Binary)  # Decommission decision

# Repurposing actions
model.coal_to_wind = Var(coal_plants.keys(), time_steps, domain=Binary)  # Repurpose decision

# Define costs
cost_of_decommissioning = sum(
    coal_plants[p] * decommissioning_costs_per_mw["coal"] * model.decom_coal[p, t]
    for p in coal_plants for t in time_steps
)

cost_of_repurposing = sum(
    coal_plants[p] * repurposing_costs_per_mw["coal"]["wind"] * model.coal_to_wind[p, t]
    for p in coal_plants for t in time_steps
)
# Define objective function
model.objective = Objective(expr=cost_of_decommissioning + cost_of_repurposing, sense=minimize)

# Constraints
# One action per plant constraint over the planning horizon
def single_action_per_plant_constraint(model, p):
    return sum(model.decom_coal[p, t] + model.coal_to_wind[p, t] for t in time_steps) <= 1
model.single_action_per_plant_constraint = Constraint(coal_plants.keys(), rule=single_action_per_plant_constraint)

# Constraint: Ensure coal energy demand is met
def coal_demand_constraint(model, t):
    return sum(
        coal_plants[p] * (1 - model.decom_coal[p, t] - model.coal_to_wind[p, t])
        for p in coal_plants
    ) >= coal_energy_demand[t]

# Constraint: Ensure wind energy demand is met
def wind_demand_constraint(model, t):
    return sum(wind_plants[w] for w in wind_plants) + sum(
        coal_plants[p] * model.coal_to_wind[p, t] for p in coal_plants
    ) >= wind_energy_demand[t]

# Constraint: Ensure total energy supply meets total demand
def total_demand_constraint(model, t):
    new_coal_capacity = sum(
        coal_plants[p] * (1 - model.decom_coal[p, t] - model.coal_to_wind[p, t])
        for p in coal_plants
    )
    new_wind_capacity = sum(wind_plants[w] for w in wind_plants) + sum(
        coal_plants[p] * model.coal_to_wind[p, t] for p in coal_plants
    )
    return (new_coal_capacity + new_wind_capacity) >= (coal_energy_demand[t] + wind_energy_demand[t])

# Apply the constraints to the model
model.coal_demand_constraint = Constraint(time_steps, rule=coal_demand_constraint)
model.wind_demand_constraint = Constraint(time_steps, rule=wind_demand_constraint)
model.total_demand_constraint = Constraint(time_steps, rule=total_demand_constraint)

# Solve the problem
solver = SolverFactory("glpk")
solver.solve(model, tee=True)